# L2 Regularization Visualizer (Interactive)

This notebook provides an interactive visualization of **L2 regularization** (Ridge regression).  
You can move the sliders to change:
- **β₁, β₂** — the coordinates of the unconstrained minimum of the loss surface  
- **R** — the radius of the L2 constraint (‖β‖₂ ≤ R)

The visualization shows:
- A **quadratic loss surface** (the “bowl”)
- The **L2 constraint circle**
- The **intersection point** where the constraint and loss contours meet

When β₁ and β₂ change, you can observe how the **optimal coefficients shift** under the L2 constraint.

---

## Dependencies

This notebook uses:
- `matplotlib` → 3D plotting  
- `ipywidgets` → interactive sliders  
- `numpy` → numerical operations  

You can install everything with:

```bash
pip install numpy matplotlib ipywidgets


# Functions

In [10]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

import ipywidgets as widgets

In [11]:
def plot_L2_constraint(a, b, R, f=lambda x, y, a, b: (x - a)**2 + (y - b)**2, resolution=150):
    """
    Visualize a convex quadratic bowl f(x,y) and an L2 constraint disk (tube).
    """
    # --- Grid setup ---
    xlim = ylim = (-2.0, 2.0)
    n = resolution

    x = np.linspace(*xlim, n)
    y = np.linspace(*ylim, n)
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y, a, b)
    z_min, z_max = Z.min(), Z.max()

    # --- Inside-disk mask: r^2 <= R^2 ---
    mask = (X**2 + Y**2) <= R**2
    Z_inside = np.where(mask, Z, np.nan)

    # --- Cylinder surface (tube) ---
    theta = np.linspace(0, 2*np.pi, 120)
    z_lin = np.linspace(z_min, z_max, 40)
    Theta, Zc = np.meshgrid(theta, z_lin)
    Xc, Yc = R * np.cos(Theta), R * np.sin(Theta)

    # --- Intersection curve: r = R ---
    xb, yb = R * np.cos(theta), R * np.sin(theta)
    zb = f(xb, yb, a, b)

    # --- Constrained minimum (projection onto disk) ---
    r_ab = np.hypot(a, b)
    if r_ab <= R:
        x_star, y_star = a, b
    else:
        scale = R / r_ab
        x_star, y_star = a * scale, b * scale
    z_star = f(x_star, y_star, a, b)

    # --- Unconstrained minimum ---
    z_uncon = f(a, b, a, b)

    # --- Plot ---
    fig = plt.figure(figsize=(9, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Surface (outside)
    ax.plot_surface(
        X, Y, Z,
        cmap='viridis', alpha=0.6,
        rstride=4, cstride=4, linewidth=0, antialiased=False
    )

    # Highlighted patch (inside tube)
    ax.plot_surface(
        X, Y, Z_inside,
        cmap='autumn', alpha=0.9,
        rstride=4, cstride=4, linewidth=0, antialiased=False
    )

    # Tube (semi-transparent)
    ax.plot_surface(Xc, Yc, Zc, alpha=0.15, linewidth=0, antialiased=False)

    # Intersection curve and base
    ax.plot(xb, yb, zb, lw=2.5, color='k')
    ax.plot(R*np.cos(theta), R*np.sin(theta), np.full_like(theta, z_min),
            lw=1.5, linestyle='--', color='k')

    # Constrained and unconstrained minima
    ax.scatter([x_star], [y_star], [z_star], s=80, color='k', marker='o')
    ax.scatter([a], [b], [z_uncon], s=70, color='k', marker='^')

    # Labels, limits, and view
    ax.set_xlabel(r'$\beta_1$')
    ax.set_ylabel(r'$\beta_2$')
    ax.set_zlabel(r'$f(\beta_1, \beta_2)$')
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_zlim(z_min, z_max)
    ax.set_box_aspect((1, 1, 0.6))
    ax.view_init(elev=28, azim=-55)

    # Legend proxies
    patch_inside = mpatches.Patch(color=plt.cm.autumn(0.6), label='Surface inside disk')
    line_inter = Line2D([], [], lw=2.5, color='k', label='Intersection curve')
    pt_constr = Line2D([], [], marker='o', linestyle='None', color='k', label='Constrained min')
    pt_uncon = Line2D([], [], marker='^', linestyle='None', color='k', label='Unconstrained min')
    tube_proxy = mpatches.Patch(alpha=0.15, label='Tube (L2 constraint)')

    ax.legend(handles=[patch_inside, line_inter, pt_constr, pt_uncon, tube_proxy], loc='upper left')
    plt.tight_layout()
    plt.show()

In [12]:
def plot_L1_constraint(a, b, t, f=lambda x,y, a, b:(x - a)**2 + (y - b)**2):
    # -----------------------------
    # Parameters (β1 ≡ x, β2 ≡ y)
    # -----------------------------
    # t = 1.0                    # L1-ball radius: |x| + |y| <= t
    # a, b = 1.5, 0.5            # unconstrained minimum location (outside the diamond if |a|+|b| > t)
    xlim = ylim = (-2.0, 2.0)  # plotting window
    n = 280                    # grid resolution

    # -----------------------------
    # Surface f(x, y)
    # -----------------------------
    # def f(x, y):
    #     return (x - a)**2 + (y - b)**2

    # -----------------------------
    # Euclidean projection onto L1 ball {v: ||v||_1 <= t}
    # (works for any dimension; here we use 2D)
    # -----------------------------
    def project_onto_l1_ball(v, radius):
        v = np.asarray(v, dtype=float)
        if np.linalg.norm(v, 1) <= radius:
            return v
        u = np.abs(v)
        if u.sum() == 0:
            return v
        sv = np.sort(u)[::-1]
        cssv = np.cumsum(sv)
        rho = np.nonzero(sv - (cssv - radius) / (np.arange(len(sv)) + 1) > 0)[0].max()
        theta = (cssv[rho] - radius) / (rho + 1.0)
        w = np.sign(v) * np.maximum(u - theta, 0)
        return w

    # Grid for the surface
    x = np.linspace(*xlim, n)
    y = np.linspace(*ylim, n)
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y, a, b)
    z_min, z_max = Z.min(), Z.max()

    # -----------------------------
    # L1 "diamond" column (prism) |x| + |y| = t extruded along z
    # Build four planar faces (one per quadrant)
    # -----------------------------
    s = np.linspace(0, t, 180)             # along the diamond edge
    z_lin = np.linspace(z_min, z_max, 60)  # vertical extent
    S, Zg = np.meshgrid(s, z_lin)

    # Face definitions:
    # Q1:  x>=0, y>=0, x + y = t  -> (x=s,   y=t-s)
    X1, Y1 = S,      t - S
    # Q4:  x>=0, y<=0, x - y = t  -> (x=s,   y=s - t)
    X2, Y2 = S,      S - t
    # Q2:  x<=0, y>=0, -x + y = t -> (x=-s,  y=t - s)
    X3, Y3 = -S,     t - S
    # Q3:  x<=0, y<=0, -x - y = t -> (x=-s,  y=s - t)
    X4, Y4 = -S,     S - t

    # -----------------------------
    # Inside-diamond mask for highlighted patch: |x| + |y| <= t
    # -----------------------------
    inside_l1 = (np.abs(X) + np.abs(Y)) <= t
    Z_inside = np.ma.masked_where(~inside_l1, Z)

    # -----------------------------
    # Intersection curve (surface ∩ diamond wall): param via angle -> L1 boundary
    # -----------------------------
    theta = np.linspace(0, 2*np.pi, 600)
    ux, uy = np.cos(theta), np.sin(theta)
    den = np.abs(ux) + np.abs(uy)
    xb = t * ux / den
    yb = t * uy / den
    zb = f(xb, yb, a, b)

    # -----------------------------
    # Constrained minimum (projection of (a,b) onto the L1 ball)
    # -----------------------------
    x_star, y_star = project_onto_l1_ball(np.array([a, b]), t)
    z_star = f(x_star, y_star, a, b)

    # Unconstrained minimum (for reference)
    z_uncon = f(a, b, a, b)

    # -----------------------------
    # Plot
    # -----------------------------
    fig = plt.figure(figsize=(9, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Full surface (light)
    surf_all = ax.plot_surface(
        X, Y, Z,
        rstride=3, cstride=3,
        cmap='viridis', alpha=0.6, linewidth=0, antialiased=True
    )

    # Highlighted surface patch inside the L1 diamond
    surf_inside = ax.plot_surface(
        X, Y, Z_inside,
        rstride=2, cstride=2,
        cmap='autumn', alpha=0.95, linewidth=0, antialiased=True
    )

    # Diamond column walls (semi-transparent)
    for Xa, Ya in [(X1, Y1), (X2, Y2), (X3, Y3), (X4, Y4)]:
        ax.plot_surface(Xa, Ya, Zg, alpha=0.18, linewidth=0, antialiased=True)

    # Intersection curve along the diamond boundary
    ax.plot(xb, yb, zb, lw=2.5, color='k')

    # Draw base/top outlines for context
    ax.plot(xb, yb, np.full_like(xb, z_min), lw=1.2, linestyle='--', color='k')
    ax.plot(xb, yb, np.full_like(xb, z_max), lw=1.0, linestyle=':', color='k')

    # Constrained and unconstrained minima
    ax.scatter([x_star], [y_star], [z_star], s=80, marker='o', color='k')
    ax.scatter([a], [b], [z_uncon], s=70, marker='^', color='k')

    # Labels & view
    ax.set_xlabel(r'$\beta_1$')
    ax.set_ylabel(r'$\beta_2$')
    ax.set_zlabel(r'$f(\beta_1,\beta_2)$')
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_zlim(z_min, z_max)
    ax.set_box_aspect((1, 1, 0.6))
    ax.view_init(elev=28, azim=-55)

    # Legend proxies (since 3D surfaces don't auto-legend cleanly)
    patch_all = mpatches.Patch(color=plt.cm.viridis(0.6), label='Surface')
    patch_inside = mpatches.Patch(color=plt.cm.autumn(0.6), label='Surface inside |β₁|+|β₂|≤t')
    line_inter = Line2D([0], [0], lw=2.5, color='k', label='Intersection curve')
    pt_constr = Line2D([0], [0], marker='o', linestyle='None', color='k', label='Constrained minimum')
    pt_uncon = Line2D([0], [0], marker='^', linestyle='None', color='k', label='Unconstrained minimum')
    wall_proxy = mpatches.Patch(alpha=0.18, label='Diamond column')

    ax.legend(handles=[patch_all, patch_inside, line_inter, pt_constr, pt_uncon, wall_proxy], loc='upper left')

    plt.tight_layout()
    plt.show()

In [13]:
def project_onto_l1_ball(v, radius):
    x, y = v
    s = abs(x) + abs(y)
    if s <= radius:
        return v
    scale = radius / s
    return np.sign(v) * np.maximum(abs(v) * scale, 0)

def plot_L1_constraint(a, b, t, f=lambda x, y, a, b: (x - a)**2 + (y - b)**2, resolution=140):
    xlim = ylim = (-2.0, 2.0)
    n = resolution

    x = np.linspace(*xlim, n)
    y = np.linspace(*ylim, n)
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y, a, b)
    z_min, z_max = Z.min(), Z.max()

    mask = (np.abs(X) + np.abs(Y)) <= t
    Z_inside = np.where(mask, Z, np.nan)

    theta = np.linspace(0, 2*np.pi, 400)
    ux, uy = np.cos(theta), np.sin(theta)
    den = np.abs(ux) + np.abs(uy)
    xb, yb = t * ux / den, t * uy / den
    zb = f(xb, yb, a, b)

    x_star, y_star = project_onto_l1_ball(np.array([a, b]), t)
    z_star = f(x_star, y_star, a, b)
    z_uncon = f(a, b, a, b)

    fig = plt.figure(figsize=(9, 7))
    ax = fig.add_subplot(111, projection='3d')

    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.6, rstride=4, cstride=4, antialiased=False)
    ax.plot_surface(X, Y, Z_inside, cmap='autumn', alpha=0.9, rstride=4, cstride=4, antialiased=False)

    S, Zg = np.meshgrid(np.linspace(0, t, 60), np.linspace(z_min, z_max, 40))
    Xq, Yq = S, t - S
    for sx, sy in [(1, 1), (1, -1), (-1, 1), (-1, -1)]:
        ax.plot_surface(sx*Xq, sy*Yq, Zg, alpha=0.18, linewidth=0, antialiased=False)

    ax.plot(xb, yb, zb, lw=2.5, color='k')
    ax.scatter([x_star], [y_star], [z_star], s=80, color='k', marker='o')
    ax.scatter([a], [b], [z_uncon], s=70, color='k', marker='^')

    ax.set_xlabel(r'$\beta_1$')
    ax.set_ylabel(r'$\beta_2$')
    ax.set_zlabel(r'$f(\beta_1,\beta_2)$')
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_zlim(z_min, z_max)
    ax.set_box_aspect((1, 1, 0.6))
    ax.view_init(elev=28, azim=-55)

    patch_inside = mpatches.Patch(color=plt.cm.autumn(0.6), label='Inside |β₁|+|β₂|≤t')
    line_inter = Line2D([], [], lw=2.5, color='k', label='Boundary curve')
    ax.legend(handles=[patch_inside, line_inter], loc='upper left')

    plt.tight_layout()
    plt.show()

In [ ]:
def l1_toy(fcn=lambda x, y, a, b: (x - a)**2 + (y - b)**2):
    """
    Create a 3D plot of the loss function and where it intersects with the L1 constraint

    Param:
        fcn : lambda function of the loss function.
    """
    a_slider = widgets.FloatSlider(value=1.5, min=-2, max=2, step=0.1, description='β₁')
    b_slider = widgets.FloatSlider(value=0.5, min=-2, max=2, step=0.1, description='β₂')
    t_slider = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.1, description='λ')

    ui = widgets.VBox([a_slider, b_slider, t_slider])
    out = widgets.interactive_output(
        lambda a, b, t: plot_L1_constraint(a, b, t, fcn), {'a': a_slider, 'b': b_slider, 't': t_slider}
    )

    display(ui, out)

In [ ]:
def l2_toy(fcn=lambda x, y, a, b: (x - a)**2 + (y - b)**2):
    """
    Create a 3D plot of the loss function and where it intersects with the L2 constraint

    Param:
        fcn : lambda function of the loss function.
    """
    # --- Sliders for parameters ---
    a_slider = widgets.FloatSlider(value=1.5, min=-2, max=2, step=0.1, description='β₁')
    b_slider = widgets.FloatSlider(value=0.5, min=-2, max=2, step=0.1, description='β₂')
    R_slider = widgets.FloatSlider(value=1.0, min=0.2, max=2.0, step=0.1, description='R')

    # --- Combine sliders in a vertical layout ---
    ui = widgets.VBox([a_slider, b_slider, R_slider])

    # --- Link sliders to interactive output ---
    out = widgets.interactive_output(
        lambda a, b, R: plot_L2_constraint(a, b, R,fcn),
        {'a': a_slider, 'b': b_slider, 'R': R_slider}
    )

    # --- Display the interactive UI ---
    display(ui, out)


# Toys

You can also adjust the function to use a different loss function, define a lambda function as a parameter to try it out!

e.g
```python
l1_toy(lambda x, y, a, b: (x - a)**2 + (y - b)**2)
```

Note: This can cause the code to fail with a bad function.

In [19]:
l1_toy()

Output()

In [ ]:
l2_toy()


Output()

In [25]:
l1_toy(lambda x,y,a,b:((x-a)*(y-b))**2/2*a*b)

Output()